# [Helm](https://helm.sh/docs/)

Helm é o gerenciador de pacotes do Kubernetes.

Já vimos como criar cada recurso separadamente. O Helm realiza a gestão dos recursos (ns, deploy, svc, ...) desta forma, com apenas um comando, podemos implantar, alterar e excluir todo nosso projeto.

In [ ]:
helm version

Para criar o helm, executamos o comando abaixo:  

```
helm create \<nome-aplicacao>
```

Este comando irá criar a estrutura abaixo:
```
<nome-aplicacao>/
├── Chart.yaml      # Informações Técnicas
├── values.yaml     # Variáveis e Valores
└── templates/      # Modelos de Deployments, Services e todos outros recursos
```

[Helm do Projeto front-app](https://github.com/kdop-dev/front-app/tree/master/helm/front-app)

## Preparando o acesso ao Ambiente

In [ ]:
export KUBECONFIG=$PWD/kubeconfig
cd /tmp/inovacao/proj
echo $PWD

## Projeto front-app

1. Publicando o Projeto front-app

    Para a publicar a aplicação via helm, podemos utilizar duas formas:
    * **helm install** - Comando para publicação da aplicação.
    * **helm upgrade** - Comando para atualização da aplicação. Passando o parâmetro **--install** ele publica a aplicação, caso não exista.

In [ ]:
#Informe um nome para o namespace
myNamespace=""
#helm install --namespace $myNamespace --create-namespace front-app front-app/helm/front-app
helm upgrade --install --namespace $myNamespace --create-namespace front-app front-app/helm/front-app

2. Listando os helms do namespace

In [ ]:
helm list --namespace $myNamespace 

3. Consultando os recursos do namespace

In [ ]:
kubectl get all -n $myNamespace 

4. Verificando o problema no POD

In [ ]:
kubectl describe pod <insert-pod-name-here> -n $myNamespace 

* O POD necessita da aplicação back-app para poder ser executada.
  * **livenessProbe**: Verifica a saúde do container. Somente se o container estiver em saudável, o POD entrará em execução.  
  Código maior ou igual a 200 e menor que 400 (200 <= StatusCode < 400) indica sucesso. Qualquer outro código indica falha.
```
        livenessProbe:
            httpGet:
              path: /health
              port: http
            initialDelaySeconds: 3 # Tempo para realizar a primeira validação
            periodSeconds: 10      # Tempo em que será realizada a sondagem da atividade.
```
  </br>
  * **readinessProbe**: Indica se o contêiner está pronto para atender às solicitações. Se a análise de prontidão falhar, o controlador de terminais removerá o endereço IP do Pod dos terminais de todos os Serviços que correspondem ao Pod. O estado padrão de prontidão antes do atraso inicial é Failure. Se um contêiner não fornecer uma sonda de prontidão, the default state is Success.


## Projeto back-app

1. Publicando o Projeto back-app

In [ ]:
helm upgrade --install --namespace $myNamespace --create-namespace back-app back-app/helm/back-app

2. Consultando os recursos do namespace

In [ ]:
kubectl get all -n $myNamespace 

3. Validando o Serviço

    Para acessar o back-app fora do Cluster, necessitamos expor temporariamente o serviço

In [ ]:
kubectl port-forward svc/back-app 8080:5000 -n $myNamespace 

Consulte o serviço acessando o link abaixo:

http://localhost:8080/health

Agora que os PODs estão em execução, podemos também validar a aplicação front-app.
Verifique no serviço do front-app o IP externo liberado para o loadbalance.

[http://\<ip-loadbalance>:8080/health](http://ip-loadbalance:8080/health)

[http://\<ip-loadbalance>:8080/index](http://ip-loadbalance:8080/index)
    


## Projeto cert-app

1. Publicando o Projeto cert-app

In [ ]:
helm upgrade --install --namespace $myNamespace --create-namespace cert-app cert-app/helm/cert-app

2. Consultando os recursos do namespace

In [ ]:
kubectl get all -n $myNamespace 

3. Validando o Serviço

    Expor o serviço temporariamente para validação

In [ ]:
kubectl port-forward svc/cert-app 8080:5000 -n $myNamespace 

Consulte o serviço acessando o link abaixo:

http://localhost:8080/health

## Excluindo as aplicações

In [ ]:
helm delete front-app --namespace $myNamespace  
helm delete back-app --namespace $myNamespace 
helm delete cert-app --namespace $myNamespace 
kubectl delete ns $myNamespace 

[<img align="left" src="media/voltar.png" width="100"/>](02_Kubernetes.ipynb)


ToDo:
* Preparar livenessProbe e readnessProbe
* Preparar ConfigMap e Secret
* Preparar resource request e limits
* Preparar HPA
* comando para o helm listar o que executou